In [1]:
#Добавим нужные библиотеки
import re
import pyshark
import nest_asyncio
nest_asyncio.apply()
import json

In [2]:
#Прочитаем пакеты из трейса и применим фильтр по протоколам sctp и s1ap
capture = pyshark.FileCapture('./test.pcap', display_filter = 'sctp and s1ap')

In [10]:
#Для более детального изучения данный блок позволяет получить все поля пакета по его номеру. 
for i in capture:
    if i.number == '34':
        print(i.s1ap._all_fields)

{'per.extension_bit': 'False', 'per.choice_index': '0', 's1ap.S1AP_PDU': '0', 's1ap.initiatingMessage_element': 'initiatingMessage', 's1ap.procedureCode': '13', 'per.enum_index': '1', 's1ap.criticality': '1', 'per.open_type_length': '55', 's1ap.value_element': 'value', 's1ap.UplinkNASTransport_element': 'UplinkNASTransport', 'per.sequence_of_length': '5', 's1ap.protocolIEs': '5', '': 'Item 0: id-MME-UE-S1AP-ID', 's1ap.ProtocolIE_Field_element': 'ProtocolIE-Field', 's1ap.id': '0', 's1ap.MME_UE_S1AP_ID': '0', 's1ap.ENB_UE_S1AP_ID': '1', 'per.octet_string_length': '13', 's1ap.NAS_PDU': '27:52:0c:ed:64:02:07:43:00:03:52:00:c2', 'nas-eps.security_header_type': '2', 'gsm_a.L3_protocol_discriminator': '0x07', 'nas-eps.msg_auth_code': '0x520ced64', 'nas-eps.seq_no': '2', 'nas-eps.nas_msg_emm_type': '0x43', 'gsm_a.len': '3', 'nas-eps.emm.esm_msg_cont': '52:00:c2', 'nas-eps.bearer_id': '5', 'nas-eps.esm.proc_trans_id': '0', 'nas-eps.nas_msg_esm_type': '0xc2', 's1ap.EUTRAN_CGI_element': 'EUTRAN-C

In [17]:
#Данный блок позволяет применить фильтры. В первую очередь происходит выборка всех пакетов и процедур, 
#которые относятся к одной транспортной sctp сессии. Далее пакеты фильтруются по номеру процедуры. Например, 12 - означает, 
#что в этом сообщении передается Initial Ue message, с помощью которого осуществляется попытка подключения к коре. 
# 13 - означает, что в данном сообщении передается Uplink Nas Transport, в котором содержится ответ об успешном/неуспешном
# завершении попытки аттача на сети. 
# Из-за невозможности применить логический фильтр с использованием библиотеки pyshark ко всем пакетам с 13 s1ap procedure
# мной было принятно решение разбирать пакеты в виде словаря, затем преобразовать его в список. И только потом, используя номер
# конкретного местоположения элемента в списке, сравнить его со значением 0x43, которое свидетельствует об успешном заверешнии
# попытки аттача в пределах одной и той же sctp сессии. Далее каждая попытка аттача добавляется в список att_att_list в виде 1 
# с указанеием sctp сессии, в список att_com_list добавляются 1 в случае успешного заверешения аттача. Таким образом, с помощью
# sctp обеспечивается совпадение между попыткой аттача и его успешным завершением. Ведь если в рамках одной sctp сессии
# абонент не получит ответ об успешном завершении попытки, то это будет означать, что попытка провалилась. 
att_att_list = []
att_com_list = []
sid_list = []
for packet in capture:
    d = packet.s1ap._all_fields
    if packet.highest_layer == 'S1AP':
        if packet.s1ap.procedurecode == '12':
            get_value = packet.sctp.data_sid
            if packet.sctp.data_sid == get_value:
                if (d['s1ap.procedureCode'] == '12'):
                    if (d['s1ap.RRC_Establishment_Cause'] == '3') and (d['nas-eps.nas_msg_emm_type'] == '0x41'):
                        print("Была попытка аттача, номер пакета ", packet.number, " номер sctp сессии ", packet.sctp.data_sid)
                        #print(packet.number)
                        #print(packet.sctp.data_sid)
                        att_att_list.append(1)
        if packet.s1ap.procedurecode == '13':
            if packet.sctp.data_sid == get_value:
                get_dict = dict(packet.s1ap._all_fields)
                keys = list(get_dict.items())
                if (keys[23] == ('nas-eps.nas_msg_emm_type', '0x43')):
                    print("Попытка аттача с sctp сессией ", packet.sctp.data_sid, " завершилась успешно, номер пакета ", packet.number)
                    #print(packet.number)
                    #print(packet.sctp.data_sid)
                    att_com_list.append(1)
print("За всё время было ", len(att_att_list), " попыток(ки) аттача")
print("Из них ", len(att_com_list), " попыток(ки) завершилось успешно")

Была попытка аттача, номер пакета  15  номер sctp сессии  0x0001
Попытка аттача с sctp сессией  0x0001  завершилась успешно, номер пакета  34
Была попытка аттача, номер пакета  44  номер sctp сессии  0x0001
Попытка аттача с sctp сессией  0x0001  завершилась успешно, номер пакета  63
За всё время было  2  попыток(ки) аттача
Из них  2  попыток(ки) завершилось успешно
